In [ ]:
# %%writefile InitializeModel.py
from MileStone_3_Create_Neural_Network.WillyAlexNet import MyAlexNet
from MileStone_3_Create_Neural_Network.WillyNet import WillyNet
from MileStone_3_Create_Neural_Network.WillyMnasNet0_5 import MyMnasNet0_5
from MileStone_3_Create_Neural_Network.WillysShuffleNet_v2_x1_0 import myShuffleNet_v2_x1_0
from torch import optim
from torch.nn import NLLLoss
import sys
sys.path.insert(0, '../')

# Neural Networks


def initializeModel(config):
    # ToDo return model from my private zoo
    modelName = config["model"]
    if modelName == "MyAlexNet":
        model = MyAlexNet()
    elif modelName == "WillyNet":
        model = WillyNet()
    elif modelName == "WillyMnasNet0_5":
        model = MyMnasNet0_5()
    elif modelName == "WillysShuffleNet_v2_x1_0":
        model = myShuffleNet_v2_x1_0()
    else:
        raise ValueError(
            f"Model {modelName} not supported or whatever something else")

    # model = WillyNet()
    optimizer = optim.SGD(
        model.parameters(),
        lr=config.get("lr", 1e-2),
        momentum=config.get("momentum", 0.5),
        weight_decay=config.get('weight_decay', 1e-6),
        nesterov=True,
    )
    criterion = NLLLoss()
    le = config['num_iters_per_epoch']
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer,
                                             step_size=le,
                                             gamma=config.get(
                                                 'gamma', 0.9)
                                             )
    return model, optimizer, criterion, lr_scheduler


In [ ]:
# %%writefile CreateTrainer.py
from ignite.engine import Engine, Events, create_supervised_evaluator
from pathlib import Path
import json
from ignite.contrib.handlers import ProgressBar
from ignite.handlers import ModelCheckpoint, Checkpoint, DiskSaver
from ignite.metrics import Accuracy, Loss, RunningAverage, ClassificationReport
from ignite.contrib.engines.common import save_best_model_by_val_score
import torch
# from ignite.metrics.precision import Precision
# from ignite.metrics.recall import Recall
# source https://colab.research.google.com/github/pytorch/ignite/blob/master/assets/tldr/teaser.ipynb#scrollTo=dFglXKeKOgdW


def create_trainer(model, optimizer, criterion, lr_scheduler, config, data):
    device = config['device']
    (train_loader, train_sampler), (valid_loader,
                                    valid_sampler), (test_loader, test_sampler) = data
    # Define any training logic for iteration update

    def train_step(engine, batch):
        x, y = batch[0].to(device), batch[1].to(device)
        model.train()
        y_pred = model(x)
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        return loss.item()

    def eval_model_on_batch(engine, batch):
        """
        Evaluation of the model on a single batch

        Args:
            engine: ignite.engine.Engine
            batch: tuple contains the training sample and their labels

        """
        model.eval()
        with torch.no_grad():
            data, target = batch
            y_pred = model(data.to(device))
            return y_pred, target.to(devtraining(config)ice)

    trainer = Engine(train_step)
    # train_evaluator = Engine(eval_model_on_batch)
    # validation_evaluator = Engine(eval_model_on_batch)
    # test_evaluator = Engine(eval_model_on_batch)

    evaluator = create_supervised_evaluator(
        model,
        metrics={"accuracy": Accuracy(),
                 #  "loss": RunningAverage(Accuracy()),
                 #  "precision": Precision(average=False),
                 #  'recall': Recall(average=False),
                 #            'cr': ClassificationReport(output_dict=True, is_multilabel=False)
                 },
        device=config.get('device', 'cpu')
    )
    # evaluatorF1 = create_supervised_evaluator(
    #     model, metrics={'cr': ClassificationReport(output_dict=True, is_multilabel=False)
    #                     },
    #     device=config.get('device', 'cpu')
    # )

    @trainer.on(Events.ITERATION_COMPLETED(every=10))
    def save_checkpoint():
        fp = Path(config.get("output_path", "./output")) / "checkpoint.pt"
        torch.save(model.state_dict(), fp)

    pbar = ProgressBar(persist=True, bar_format="")

    # Run model evaluation every 3 epochs and show results
    @trainer.on(Events.EPOCH_COMPLETED(every=1))
    def evaluate_model():
        state = evaluator.run(valid_loader)
        metrics = state.metrics
    #     # acc = metrics['accuracy']
    #     # loss = metrics['loss']
    #     # train_evaluator.run(train_loader)
    #     # validation_evaluator.run(valid_loader)

    # @trainer.on(Events.EPOCH_COMPLETED(every=5))
    # def F1ScoreEval(engine):
    #     state = evaluatorF1.run(valid_loader)
    #     toPrint = ""
    #     res_dic = state.metrics['cr']
    #     for idx, key in enumerate(res_dic):
    #         temp = "Class: {} f1-score: {:.2f}, ".format(
    #             key, res_dic[key]['f1-score'])
    #         if idx == 4 or idx == 8 or idx == 12 or idx == 16:
    #             toPrint += '\n'
    #         if key == 'macro avg':
    #             temp = "\nMacro Average:\nPrecision: {:.3f}, Recall: {:.3f}, F1 {:.3f}".format(
    #                 res_dic[key]['precision'], res_dic[key]['recall'], res_dic[key]['f1-score'],)
    #         toPrint += temp
    #     pbar.log_message("Scores:\n{}".format(toPrint))

    RunningAverage(output_transform=lambda x: x).attach(
        trainer, 'loss'
    )
    # pbar.attach(trainer, ['loss'])
    pbar.attach(trainer, output_transform=lambda x: {"batch loss": x})

    # Accuracy(output_transform=lambda x: x).attach(train_evaluator, 'train_acc')
    # Loss(criterion).attach(train_evaluator, 'train_NLLLoss')

    # Accuracy(output_transform=lambda x: x).attach(
    #     validation_evaluator, 'val_acc')
    # Loss(criterion).attach(validation_evaluator, 'val_NLLLoss')

    # checkpointer = ModelCheckpoint(
    #     config.get('checkpointPath', './tmp/models'), config.get('model',
    #                                                              'Unknown_you_lazy_bastart'),
    #     n_saved=config.get('max_saved_checkout', 1), create_dir=True,
    #     save_as_state_dict=True, require_empty=config.get('require_empty', True)
    # )
    # trainer.add_event_handler(Events.EPOCH_COMPLETED(every=3),
    #                           checkpointer, {config.get('model'): model}
    #                           )

    # to_save = {'trainer': trainer,
    #            'model': model,
    #            'optimizer': optimizer,
    #            'lr_scheduler': lr_scheduler,
    #            }

    # handler = Checkpoint(to_save,
    #                      DiskSaver(config['trainer_save_path'],
    #                                create_dir=True, require_empty=config['require_empty']
    #                                )
    #                      )
    # bestModelSaver = save_best_model_by_val_score(output_path=config['best_model_path'],
    #                                               evaluator=evaluator, model=model,
    #                                               metric_name='accuracy',
    #                                               n_saved=config.get(
    #     "best_n_saved", 2)
    # )
    # evaluator.add_event_handler(Events.EPOCH_COMPLETED, bestModelSaver)

    # trainer.add_event_handler(Events.EPOCH_COMPLETED, handler)

    # return trainer, train_evaluator, validation_evaluator, test_evaluator, evaluator, pbar
    return trainer, evaluator


In [ ]:
%%writefile TrainingIgnite.py
from InitializeModel import initializeModel
from CreateTrainer import create_trainer
from getData import getData
#from ignite.engine import create_supervised_evaluator, Events
# from ignite.metrics import Accuracy, Loss, RunningAverage
from ignite.contrib.engines import common
# from ignite.handlers import EarlyStopping
# from ignite.metrics import ClassificationReport
import ignite.distributed as idist
# import torch

# source https://colab.research.google.com/github/pytorch/ignite/blob/master/assets/tldr/teaser.ipynb#scrollTo=dFglXKeKOgdW
# dependenciues

# ignite

# PyTorch


def training(config):
    # Setup dataflow and
    data = getData(config)
    model, optimizer, criterion, lr_scheduler = initializeModel(config)
    model.to(config.get('device', 'cpu'))
    # Setup model trainer and evaluator
    trainer, evaluator = create_trainer(model, optimizer, criterion,
                                        lr_scheduler, config, data
                                        )
    data_loader, _, _ = data
    train_loader, train_sampler = data_loader

    if config['train_now'] is not True:
        return trainer, model, optimizer, criterion, lr_scheduler, train_loader

    if idist.get_rank() == 0:
        # print('Start logging', config['tensorboard_logger_path'])
        tb_logger = common.setup_tb_logging(
            output_path=config.get("tensorboard_logger_path", "output"),
            trainer=trainer, optimizers=optimizer,
            evaluators={"validation": evaluator,
                        # 'cr': evaluator,
                        # 'loss': trainer,
                        # 'val_NLLLoss': validation_evaluator,
                        # 'val_acc': validation_evaluator,
                        # 'train_NLLLoss': train_evaluator,
                        # 'train_acc': train_evaluator,
                        },
            log_every_iters=1,
        )

    trainer.run(train_loader, max_epochs=config.get("max_epochs", 3))

    if idist.get_rank() == 0:
        # print("cloosing tb_logger...")
        tb_logger.close()
    # tb_logger.close()
    # return trainer, (train_loader, train_sampler), (valid_loader,
    #                                                 valid_sampler), (test_loader, test_sampler)


In [1]:
def initConfig(model_name: str) -> dict:
    from datetime import date, datetime
    datestamp = date.today().strftime("%d_%m_%Y")
    timestamp = datetime.now().strftime("%H")
    from torch import device
    config = {
        "model": f"{model_name}",
        "dataset": "Willys",
        'batch_size': 400,
        "valid_procentage": 0.15,
        "test_procentage": 0.1,
        'data_path': "../Dataset_Willys_2020/ORGINAL/",
        'output_path': f'./training_files/training_Checkpoints/{model_name}/date_{datestamp}/time_{timestamp}/checkpoints/',
        'tensorboard_logger_path': f'./training_files/tensorboard/{model_name}/date_{datestamp}/time_{timestamp}/log',
        'device': device('cuda:0'),
        'checkpointPath': f'./training_files/training_Checkpoints/{model_name}/date_{datestamp}/time_{timestamp}/checkpoints/',
        "require_empty": False,
        "max_epochs": 15,
        "lr": 1e-2,
        "momentum": 0.5,
        "weight_decay": 1e-6,
        "gamma": 0.9,
        "best_model_path": f'./training_files/bestModels/{model_name}/date_{datestamp}/time_{timestamp}/bestModel/',
        'trainer_save_path': f'./training_files/training_Checkpoints/{model_name}/date_{datestamp}/time_{timestamp}/trainer/',
        'train_now': True
    }
    return config


In [2]:
modelNamesDict = {'AlexNet': "MyAlexNet",
                  'MustafaNet': "WillyNet",
                  'MnasNet0_5': "WillyMnasNet0_5",
                  'ShuffleNet_V2_x1_0': "WillysShuffleNet_v2_x1_0",
                  }


In [ ]:
# %reload_ext tensorboard
%load_ext tensorboard
# %tensorboard - -logdir = config['tensorboard_logger_path']


In [3]:
from TrainingIgnite import training
config = initConfig(modelNamesDict['MustafaNet'])
config["max_epochs"] = 100
config["train_now"] = True
config['batch_size'] = 800
# config["lr"] = 10e-1
# config["momentum"] = 9e-1
# config["gamma"] = 9.5e-1
print(config["model"], config['train_now'])


WillyNet True


In [4]:
training(config)

../Dataset_Willys_2020/ORGINAL/
total number of images 9730, which 7444 of them is used for training
And 1459 for validation and 827 for testing


Epoch [20/20]: 100%|██████████| 9/9 [00:52<00:00,  6.55s/it, batch loss=2.22]


In [4]:
training(config)

../Dataset_Willys_2020/ORGINAL/
total number of images 9730, which 7444 of them is used for training
And 1459 for validation and 827 for testing


Epoch [100/100]: 100%|██████████| 9/9 [00:52<00:00,  6.56s/it, batch loss=2.12]


In [5]:
# from TrainingIgnite import training
#config = initConfig(modelNamesDict['MustafaNet'])
config["max_epochs"] = 180
config["train_now"] = True
config['batch_size'] = 400
config["lr"] = 10e-1
config["momentum"] = 9e-1
config["gamma"] = 9.5e-1
print(config["model"], config['train_now'])
training(config)


WillyNet True
../Dataset_Willys_2020/ORGINAL/
total number of images 9730, which 7444 of them is used for training
And 1459 for validation and 827 for testing


Epoch [180/180]: 100%|██████████| 18/18 [00:50<00:00,  2.99s/it, batch loss=2.02]


In [ ]:

training(config)

In [ ]:
training(config)

In [ ]:
from CreateTrainer import create_trainer
# trainer, train_evaluator, validation_evaluator, test_evaluator, evaluator, pbar 
# Setup model trainer and evaluator
whatever = create_trainer(model, optimizer, criterion, lr_scheduler, config, data)
len(whatever)

In [ ]:
# Setup model trainer and evaluator
trainer, train_evaluator, validation_evaluator, test_evaluator, evaluator, pbar = create_trainer(
    model, optimizer, criterion, lr_scheduler, config, data
)

In [ ]:
trainer, model, optimizer, criterion, lr_scheduler, train_loader = training(
    config)


In [ ]:
import torch
to_load = {'trainer': trainer, 'model': model,
           'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
checkpoint = torch.load(config['trainer_save_path']+'checkpoint_90.pt')

In [ ]:
from ignite.handlers import Checkpoint
Checkpoint.load_objects(to_load=to_load, checkpoint=checkpoint)

In [ ]:
trainer.run(train_loader,max_epochs=10)

In [ ]:
test = {"0": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0}, "1": {"precision": 0.5813953488372093, "recall": 0.5434782608695652, "f1-score": 0.5617977528089882}, "2": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0}, "3": {"precision": 0.2714285714285714, "recall": 0.3392857142857143, "f1-score": 0.30158730158730107}, "4": {"precision": 0.4794520547945205, "recall": 0.42168674698795183, "f1-score": 0.44871794871794823}, "5": {"precision": 0.208955223880597, "recall": 0.2222222222222222, "f1-score": 0.21538461538461487}, "6": {"precision": 0.75, "recall": 0.16666666666666666, "f1-score": 0.27272727272727243}, "7": {"precision": 0.5263157894736842, "recall": 0.2857142857142857, "f1-score": 0.3703703703703699}, "8": {"precision": 0.3111111111111111, "recall": 0.3218390804597701, "f1-score": 0.3163841807909599}, "9": {"precision": 0.2975206611570248,
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 "recall": 0.576, "f1-score": 0.392370572207084}, "10": {"precision": 0.47297297297297297, "recall": 0.5511811023622047, "f1-score": 0.5090909090909086}, "11": {"precision": 0.42105263157894735, "recall": 0.4, "f1-score": 0.41025641025640974}, "12": {"precision": 0.4117647058823529, "recall": 0.42857142857142855, "f1-score": 0.4199999999999995}, "13": {"precision": 0.4308510638297872, "recall": 0.574468085106383, "f1-score": 0.49240121580547064}, "14": {"precision": 0.6923076923076923, "recall": 0.28378378378378377, "f1-score": 0.40255591054313056}, "15": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0}, "16": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0}, "17": {"precision": 0.0, "recall": 0.0, "f1-score": 0.0}, "macro avg": {"precision": 0.3252848792919151, "recall": 0.28416096539055424, "f1-score": 0.2840913589050254}}
print(type(test))


In [ ]:
trainer = create_trainer(model, optimizer, criterion, lr_scheduler, config)


In [ ]:
import torchinfo
import torch
from MileStone_3_Create_Neural_Network.WillyAlexNet import MyAlexNet
#willyAlexNet = MyAlexNet()
model = MyAlexNet()
device = torch.device('cuda:0')
model.to(device)
print(model)
args = {"lr": 0.01, "momentum":0.5, "epochs": 5, "log_interval": 3 }
optimizer = torch.optim.SGD(model.parameters(), lr=args["lr"], momentum = args["momentum"])
# optimizer = optim.AdamW(model.parameters(), weight_decay=1e-6)
criterion = torch.nn.NLLLoss()

In [ ]:
# Single operation step when training
def train_one_step(engine, batch):
    """
    Single operation step when training

    Args:
        engine: ignite.engine.Engine
        batch: tuple contains the training sample and their labels
    """
    model.train(True)
    optimizer.zero_grad()
    data, target = batch
    output = model(data.to(device))
    loss = criterion(output, target.to(device))
    loss.backward()
    optimizer.step()
    
    return loss.item()
            

In [ ]:
def eval_model_on_batch(engine, batch):
    """
    Evaluation of the model on a single batch

    Args:
        engine: ignite.engine.Engine
        batch: tuple contains the training sample and their labels

    """
    model.eval()
    with torch.no_grad():
        data, target = batch
        y_pred = model(data.to(device))
        
        return y_pred, target.to(device)

In [ ]:
trainer = Engine(train_one_step)
train_evaluator = Engine(eval_model_on_batch)
validation_evaluator = Engine(eval_model_on_batch)

In [ ]:
RunningAverage(output_transform=lambda x: x).attach(trainer,'loss')

In [ ]:
Accuracy(output_transform=lambda x: x).attach(train_evaluator,'accuracy')
Loss(criterion).attach(train_evaluator, 'NLLLoss')

In [ ]:
Accuracy(output_transform=lambda x: x).attach(validation_evaluator,'accuracy')
Loss(criterion).attach(validation_evaluator, 'NLLLoss')

In [ ]:
pbar = ProgressBar(persist=True, bar_format="")
pbar.attach(trainer,['loss'])

In [ ]:
def score_function(engine):
    val_loss = engine.state.metrics['NLLLoss']
    return -val_loss

handler_earlyStopping = EarlyStopping(patience=5, score_function=score_function, trainer=trainer)
validation_evaluator.add_event_handler(Events.COMPLETED,handler_earlyStopping)

In [ ]:
@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    train_evaluator.run(train_loader)
    metrics = train_evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_nll_loos = metrics['NLLLoss']
    pbar.log_message(
        "Training Results - Epoch: {} Avg accuracy: {:.2f} avg loss {:.2f}".format(engine.state.epoch, avg_accuracy,
       avg_nll_loos)
       )

def log_validation_results(engine):
    validation_evaluator.run(valid_loader)
    metrics = validation_evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_nll_loos = metrics['NLLLoss'] 
    pbar.log_message
    (
        "Training Results - Epoch: {} Avg accuracy: {:.2f} avg loss {:.2f}".format(engine.state.epoch, avg_accuracy, avg_nll_loos)
    )
    pbar.n = pbar.last_print_n = 0


trainer.add_event_handler(Events.EPOCH_COMPLETED, log_validation_results)

In [ ]:
checkpointer = ModelCheckpoint('./tmp/models', 'willyAlexNet', n_saved=2, create_dir=True, save_as_state_dict=True)
trainer.add_event_handler(Events.EPOCH_COMPLETED, checkpointer, {'willyAlexNet': model})

In [ ]:
trainer.run(train_loader, max_epochs=30)

In [ ]:
trainer

In [ ]:
from pathlib import Path


def create_trainer(model, optimizer, criterion, lr_scheduler, config):
    device = config['device']
    # Define any training logic for iteration update

    def train_step(engine, batch):
        x, y = batch[0].to(device), batch[1].to(device)
        model.train()
        y_pred = model(x)
        loss = criterion(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

        return loss.item()

    # Define trainer engine
    trainer = Engine(train_step)

    @trainer.on(Events.ITERATION_COMPLETED(every=3))
    def save_checkpoint():
        fp = Path(config.get("output_path", "output")) / "checkpoint.pt"
        torch.save(model.state_dict(), fp)

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_training_results(engine):
        train_evaluator.run(train_loader)
        metrics = train_evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll_loos = metrics['NLLLoss']
        pbar.log_message(
            "Training Results - Epoch: {} Avg accuracy: {:.2f} avg loss {:.2f}".format(engine.state.epoch, avg_accuracy,
                                                                                       avg_nll_loos)
        )

    @trainer.on(Events.EPOCH_COMPLETED)
    def log_validation_results(engine):
        validation_evaluator.run(valid_loader)
        metrics = validation_evaluator.state.metrics
        avg_accuracy = metrics['accuracy']
        avg_nll_loos = metrics['NLLLoss']
        pbar.log_message
        (
            "Training Results - Epoch: {} Avg accuracy: {:.2f} avg loss {:.2f}".format(
                engine.state.epoch, avg_accuracy, avg_nll_loos)
        )
        pbar.n = pbar.last_print_n = 0

    # Add progress bar showing batch loss value
    ProgressBar().attach(trainer, output_transform=lambda x: {"batch loss": x})
    checkpointer = ModelCheckpoint(
        config.get('checkpointPath', './tmp/models'), config.get('model',
                                                                 'Unknown_you_lazy_bastart'),
        n_saved=config.get('max_saved_checkout', 2), create_dir=True,
        save_as_state_dict=True
    )
    trainer.add_event_handler(Events.EPOCH_COMPLETED,
                              checkpointer, {config.get('model'): model}
                              )

    return trainer


In [ ]:
from MileStone_3_Create_Neural_Network.WillyAlexNet import MyAlexNet
import torch


def initalizeModel(config):
    # ToDo return model from my private zoo
    model = MyAlexNet()
    optimizer = torch.optim.SGD(
        model.parameters(),
        lr=config.get("lr", 1e-2),
        momentum=config.get("momentum", 0.5),
        weight_decay=config.get('weight_decay', 1e-6),
        nesterov=True,
    )
    criterion = torch.nn.NLLLoss()
    le = config['num_iter_per_epoch']
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer=optimizer,
                                                   step_size=le,
                                                   gamma=config.get(
                                                       'gamma', 0.9)
                                                   )
    return model, optimizer, criterion, lr_scheduler


In [ ]:
import torchinfo
config = {
    "model": "MyAlexNet",
    "dataset": "Willys",
    'batch_size': 400,
    "num_iter_per_epoch": len(train_loader),
    'data_path': "../Dataset_Willys_2020/ORGINAL/",
    'output_path': './tmp/output/test',
    'device': torch.device('cuda:0')
}
model, optimizer, criterion, lr_scheduler = initalizeModel(config=config)
torchinfo.summary(model);


In [ ]:
trainer = create_trainer(model, optimizer, criterion, lr_scheduler, config)


In [ ]:
from pathlib import Path
Path(config.get('data_path', "dd")) / "checkpointer.pt"


In [ ]:
import torchinfo
import torch
from MileStone_3_Create_Neural_Network.WillyAlexNet import MyAlexNet
#willyAlexNet = MyAlexNet()
model = MyAlexNet()
device = torch.device('cuda:0')
model.to(device)
print(model)
args = {"lr": 0.01, "momentum":0.5, "epochs": 5, "log_interval": 3 }
optimizer = torch.optim.SGD(model.parameters(), lr=args["lr"], momentum = args["momentum"])
# optimizer = optim.AdamW(model.parameters(), weight_decay=1e-6)
criterion = torch.nn.NLLLoss()

In [ ]:
import torchinfo
import torch
from MileStone_3_Create_Neural_Network.WillyAlexNet import MyAlexNet
#willyAlexNet = MyAlexNet()
model = MyAlexNet()
device = torch.device('cuda:0')
model.to(device)
print(model)

In [ ]:
import torch.optim as optim
def train(model, device, train_loaderIn, args):
    optimizer = optim.SGD(model.parameters(), lr=args["lr"], momentum = args["momentum"])
    # optimizer = optim.AdamW(model.parameters(), weight_decay=1e-6)
    for epoch in range(1, args["epochs"]+1):
        train_epoch(epoch=epoch, args=args, model=model,
         device=device, data_loader=train_loaderIn, optimizer=optimizer
         )


In [ ]:
from torch.nn import functional as F
def train_epoch(epoch:int, args:dict, model, device, data_loader, optimizer):
    model.train(True)
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        # output_pred = F.log_softmax(output,dim=1)
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args["log_interval"] == 0:
            print("{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()
                ) 
            )
            

In [ ]:
def test(model, device, test_loaderIn):
    test_epoch(model=model, device=device,data_loader=test_loaderIn)
    

In [ ]:
def test_epoch(model, device, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data.to(device))
            # output_pred = F.log_softmax(output,dim=1)
            test_loss += F.nll_loss(output, target.to(device),reduction='sum').item()
            pred = output.max(1)[1]
            correct += pred.eq(target.to(device)).sum().item()

    test_loss /=  len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset))
        )


In [ ]:
#import the model
from MileStone_3_Create_Neural_Network.WillyNet import WillyNet
import torch
assert torch.cuda.is_available()
torch.backends.cudnn.benchmark=True

willy = WillyNet()
from torchinfo import summary
summary(willy)




In [ ]:
args = {"lr": 0.01, "momentum":0.5, "epochs": 5, "log_interval": 3 }
device = torch.device('cuda:0')
willy.to(device)
willy.share_memory()
train(model=willy, device=device, train_loader=train_loader,args=args)


In [ ]:
test(model=willy, device=device, test_loader=train_loader)


In [ ]:
import os
import torch
import torch.optim as optim
import torch.nn.functional as F


def train(rank, args, model, device, dataset, dataloader_kwargs):
    train_loader = torch.utils.data.DataLoader(dataset, **dataloader_kwargs)

    optimizer = optim.SGD(model.parameters(), lr=args.lr,
                          momentum=args.momentum)
    for epoch in range(1, args.epochs + 1):
        train_epoch(epoch, args, model, device, train_loader, optimizer)


def test(args, model, device, dataset, dataloader_kwargs):
    torch.manual_seed(args.seed)
    test_loader = torch.utils.data.DataLoader(dataset, **dataloader_kwargs)
    test_epoch(model, device, test_loader)


def train_epoch(epoch, args, model, device, data_loader, optimizer):
    model.train()
    pid = os.getpid()
    for batch_idx, (data, target) in enumerate(data_loader):
        optimizer.zero_grad()
        output = model(data.to(device))
        loss = F.nll_loss(output, target.to(device))
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('{}\tTrain Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                pid, epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
            if args.dry_run:
                break


def test_epoch(model, device, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            output = model(data.to(device))
            # sum up batch loss
            test_loss += F.nll_loss(output, target.to(device),
                                    reduction='sum').item()
            pred = output.max(1)[1]  # get the index of the max log-probability
            correct += pred.eq(target.to(device)).sum().item()

    test_loss /= len(data_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))
